<a href="https://colab.research.google.com/github/gopichandchalla16/infosys-internship-real-time-industry-insight-system/blob/janmejay-dev/Realtime_Industry_Insight_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install wikipedia --quiet

In [ ]:
import openai, tweepy, pandas as pd, matplotlib.pyplot as plt

In [ ]:
print("✅ Environment ready!")

✅ Environment ready!


In [1]:
# Testing completed

Testing completed on colab


Infosys Project : Real-Time Industry Insight & Strategic Intelligence System

Sprint 1 – Data Sourcing & Handling


Objective :: To collect real-time data from external APIs, preprocess it, perform sentiment analysis using a pre-trained Hugging Face model, and visualize the results through a simple interactive dashboard.

 Step 1 :: Install Dependencies

In [2]:
!pip install wikipedia yfinance feedparser textblob transformers matplotlib --quiet


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.1 MB/s eta 0:00:00


⚙️ Step 1 – Imports

In [49]:
import pandas as pd
import wikipedia
import yfinance as yf
import feedparser
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
from transformers import pipeline
from newsapi import NewsApiClient

In [48]:
!pip install newsapi-python --quiet

In [4]:
import plotly.express as px
import plotly.graph_objects as go

💬 Step 2 – Get User Input

In [50]:
company = input("Enter the company name: ").strip()
print(f"\n📊 Analyzing data for: {company}\n")

Enter the company name: netflix

📊 Analyzing data for: netflix



Fetch Summary from Wikipedia

In [51]:
try:
    # fetch the first 3 sentences of the summary
    summary = wikipedia.summary(company, sentences=3)
    print(f"\n📘 Wikipedia Summary for {company}:\n")
    print(summary)
except wikipedia.DisambiguationError as e:
    print("⚠️ The name is ambiguous. Try one of these options:")
    print(e.options[:10])
except wikipedia.PageError:
    print("❌ No page found for that company name.")
except Exception as err:
    print("⚠️ Unexpected error:", err)


# extra feature

try:
    page = wikipedia.page(company)
    print("\n🔗 URL:", page.url)
    print("\n📑 Title:", page.title)
    print("\n📝 First 500 characters:\n")
    print(page.content[:500])
except Exception as err:
    print("⚠️ Could not fetch full page:", err)




📘 Wikipedia Summary for netflix:

Netflix is an American subscription video on-demand over-the-top streaming service. The service primarily distributes original and acquired films and television shows from various genres, and it is available internationally in multiple languages.
Launched in 2007, nearly a decade after Netflix, Inc.

🔗 URL: https://en.wikipedia.org/wiki/Netflix

📑 Title: Netflix

📝 First 500 characters:

Netflix is an American subscription video on-demand over-the-top streaming service. The service primarily distributes original and acquired films and television shows from various genres, and it is available internationally in multiple languages.
Launched in 2007, nearly a decade after Netflix, Inc. began its pioneering DVD-by-mail movie rental service, Netflix is the most-subscribed video on demand streaming media service, with 301.6 million paid memberships in more than 190 countries as of 202


In [53]:
API_KEY = "1e2314f549e24f26a8335db1186ce244"
newsapi = NewsApiClient(api_key=API_KEY)

# --- Step 2: Ask for Company Name ---
company = input("Enter company/topic name (e.g., Tesla, Infosys, Apple): ").strip()

# --- Step 3: Fetch News Articles ---
try:
    articles = newsapi.get_everything(
        q=company,
        language='en',
        sort_by='publishedAt',
        page_size=25
    )

    # Check if any results returned
    if not articles["articles"]:
        print(f"⚠️ No news found for {company}. Try another keyword.")
    else:
        # Create DataFrame
        news_list = [
            [a["title"] + " " + (a["description"] or ""), "Google News"]
            for a in articles["articles"]
        ]
        df_news = pd.DataFrame(news_list, columns=["text", "source"])
        print(f"✅ News data fetched successfully for {company}! ({len(df_news)} articles)\n")

        # --- Step 4: Display News ---
        pd.set_option('display.max_colwidth', None)
        display(df_news.head(10))

except Exception as e:
    print(f"❌ Error fetching news: {e}")

Enter company/topic name (e.g., Tesla, Infosys, Apple): tesla
✅ News data fetched successfully for tesla! (25 articles)



,text,source
0,"Altman touts trillion-dollar AI vision as OpenAI restructures to chase scale SAN FRANCISCO -Soon after ChatGPT was released to the public in late 2022, OpenAI CEO Sam Altman told employees they were on the cusp of a new technological revolution. OpenAI could soon become ""the most important company in the history of Silicon Valley,""…",Google News
1,"[Remanufactured]Ottocast Carplay Adapter $32.64 (a $49.50) Delivered @ Krse_19 eBay Remanufactured ✅ KEY FEATURES 2-in-1 Wireless CarPlay + HDMI Screen Mirroring Universal Compatibility Easy Plug & Play Includes HDMI to Type-C Cable Excludes: Tesla, BMW, Honda …",Google News
2,"DeepSeek may have found a new way to improve AI’s ability to remember An AI model released by Chinese AI company DeepSeek uses new techniques that could significantly improve AI’s ability to “remember.” Released last week, the optical character recognition (OCR) model works by extracting text from an image and turning it into m…",Google News
3,"Saudi Arabia is now a nation of city-dwellers, and ‘cities don’t lie,’ says urban development leader Also: All the news and watercooler chat from Fortune.",Google News
4,"Preeti Jhangiani on 25 years of Mohabbatein: To be remembered as the ‘Mohabbatein girl’ is an honour I carry with love Preeti Jhangiani looks back on 25 years of Mohabbatein with nostalgia, calling it a film that defined youthful love and continues to live in people’s hearts. The actress says being remembered as the ‘Mohabbatein girl’ is an honour she carries with pride and g…",Google News
5,"Zodiac Signs that Never Show Gratitude Expressing gratitude is a vital personality trait, yet some individuals struggle to convey their appreciation. This article highlights two zodiac signs, Capricorn and Scorpio, who find it particularly difficult to say 'thank you' or offer words of appreciatio…",Google News
6,Nato airspace: Can Baltic startups counter Russian drones? Europe News: What would happen if Russia sent hundreds of drones at once into Nato airspace?This nightly reality in Russia's war in Ukraine is a scenario the EU an.,Google News
7,"The many mysterious meanings of black cats Environment News: It's the darkest time of the year. You're out walking, the sun has already gone down. Suddenly, out of the shadows, a black cat darts across your path.",Google News
8,"Little boy stuck in elevator prays to God and what happens next seems straight out of a dream A heartwarming viral video shows a young boy, trapped alone in an elevator, reciting a prayer for help. His innocent faith was met with a miraculous opening of the doors, inspiring viewers worldwide. The clip serves as a powerful reminder of the resilience of…",Google News
9,"Text messages told employees they had been laid off: How Amazon's job cuts unfolded According to people familiar with the matter, this rather unorthodox strategy of laying off employees was to ensure that people did not show up to work to find that their access badges no longer functioned.",Google News


No charts were generated by quickchart


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_2['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

🌐 Step 3 – Collect Real-Time Data

In [54]:
# Wikipedia
try:
    summary = wikipedia.summary(company, sentences=3)
except:
    summary = "No Wikipedia summary found."

# Yahoo Finance
try:
    ticker = yf.Ticker(company)
    market_data = ticker.history(period="1mo")[["Close"]]
    if market_data.empty:
        raise ValueError
except:
    print("⚠️ Could not fetch Yahoo Finance data; using TSLA as fallback.")
    market_data = yf.Ticker("TSLA").history(period="1mo")[["Close"]]

# Google News RSS
feed = feedparser.parse(f"https://news.google.com/rss/search?q={company}")
headlines = [entry.title for entry in feed.entries]

# Synthetic tweets
tweets = [f"{company} is doing great! Love the innovation #{i}" for i in range(25)] + \
         [f"{company} is facing issues lately #{i}" for i in range(25)]

print("✅ Data collected successfully.")


ERROR:yfinance:$TESLA: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")


⚠️ Could not fetch Yahoo Finance data; using TSLA as fallback.
✅ Data collected successfully.


🧹 Step 4 – Pre-Process

In [55]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^A-Za-z\s]", "", text)
    return text.lower().strip()

data = {
    "Source": ["Wikipedia"] + ["News"] * len(headlines) + ["Tweet"] * len(tweets),
    "Text": [summary] + headlines + tweets
}
df = pd.DataFrame(data)
df["Cleaned_Text"] = df["Text"].apply(clean_text)


🤖 Step 5 – Sentiment Analysis (TextBlob for speed)

In [56]:
df["Polarity"] = df["Cleaned_Text"].apply(lambda x: TextBlob(x).sentiment.polarity)
df["Sentiment"] = df["Polarity"].apply(
    lambda p: "Positive" if p > 0 else "Negative" if p < 0 else "Neutral"
)


📊 Step 6 – Interactive Visualizations (Plotly)

① Sentiment Distribution

In [57]:
sent_counts = df["Sentiment"].value_counts().reset_index()
sent_counts.columns = ["Sentiment", "Count"]

fig1 = px.bar(
    sent_counts,
    x="Sentiment", y="Count", color="Sentiment",
    title="Sentiment Distribution", text="Count",
    color_discrete_map={"Positive":"green","Negative":"red","Neutral":"gray"}
)
fig1.update_layout(xaxis_title="Sentiment Type", yaxis_title="Number of Mentions")
fig1.show()


② Market Price Trend

In [58]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=market_data.index, y=market_data["Close"],
    mode="lines+markers", name="Market Price",
    line=dict(width=2)
))
fig2.update_layout(
    title=f"{company} Market Price Trend (Last 30 Days)",
    xaxis_title="Date", yaxis_title="Price (USD)", hovermode="x"
)
fig2.show()

③ Tweet Sentiment Trend

In [59]:
tweets_df = df[df["Source"]=="Tweet"].copy()
tweets_df["Index"] = range(1, len(tweets_df)+1)

fig3 = px.line(
    tweets_df, x="Index", y="Polarity",
    title="Sentiment Trend Across Tweets",
    markers=True, line_shape="spline"
)
fig3.update_layout(xaxis_title="Tweet Index", yaxis_title="Sentiment Polarity (-1 to 1)")
fig3.show()

④ Sentiment by Source (Extra Insight)

In [60]:
fig4 = px.box(
    df, x="Source", y="Polarity", color="Source",
    title="Sentiment Distribution by Data Source"
)
fig4.update_traces(boxmean=True)
fig4.show()

📈 Step 7 – Summary Output

In [61]:
print("📊 Company Analyzed :", company)
print("📰 Wikipedia Summary :", summary)
print(f"😊 Average Sentiment Score : {df['Polarity'].mean():.3f}")
print(f"✅ Positive Tweets     : {(df['Sentiment']=='Positive').mean()*100:.1f}%")
print(f"❌ Negative Tweets     : {(df['Sentiment']=='Negative').mean()*100:.1f}%")


📊 Company Analyzed : tesla
📰 Wikipedia Summary : No Wikipedia summary found.
😊 Average Sentiment Score : 0.103
✅ Positive Tweets     : 40.0%
❌ Negative Tweets     : 25.3%


Alpha Vantage testing


Alpha Vantage is a robust API for financial data.

In [29]:
!pip install alpha_vantage --quiet


In [35]:
!pip install yfinance --quiet

Import neccessary file and input taken

USER INPUT


Fetch time‐series data from the alpha vantage